
# Lab-03 — A Tiny Neural Network
**AI Demystified: Decoding Models, Compute, and Connectivity**

Welcome! This lab walks you through **building** and **training** a very **small** **neural** **network**. You will see how data flows forward, how gradients flow backward and how learning happens step by step.


**Problem & intuition.** We’ll teach a tiny neural net to perform a **majority-of-3** decision: given three binary inputs $(x_1, x_2, x_3)$, predict **1** if **at least two** are 1, else **0**.  
**Formal rule:**
$$
y =
\begin{cases}
1 & \text{if } x_1 + x_2 + x_3 \ge 2,\\
0 & \text{otherwise.}
\end{cases}
$$
**Analogy:** *Sensor voting* — motion, sound, light; alert if **two or more** agree.  
We’ll **train on six patterns** and keep **two unseen** for a quick generalization check. With Keras, it’s simply: **build → compile → fit → evaluate**.

**Data framing**
- Inputs **X**: each example has 3 binary features → shape $(m, 3)$ where $m$ is the number of training examples  
- Label **y**: majority target (0/1) → shape $(m, 1)$  
- Goal: learn a function $f:\{0,1\}^3 \to \{0,1\}$ such that $\hat y = f(x_1,x_2,x_3)$ and ideally $\hat y \approx \mathbf{1}[\,x_1 + x_2 + x_3 \ge 2\,]$.

**Intuition with concrete examples.** The label is the **majority** of the three bits (1 if \(x_1+x_2+x_3 \ge 2\), else 0).

**Training set (6 patterns):**
- [0, 0, 1] → 0
- [0, 1, 0] → 0
- [1, 0, 0] → 0
- [0, 1, 1] → 1
- [1, 0, 1] → 1
- [1, 1, 1] → 1

**Unseen test set (2 patterns):**
- [0, 0, 0] → 0
- [1, 1, 0] → 1





## 1) Setup

In [ ]:
import tensorflow as tf

In [ ]:
print(tf.__version__)

In [ ]:
tf.random.set_seed(42)

## 2) Data — all 8 patterns; labels by the majority rule

In [ ]:

X_all = tf.constant([
    [0.,0.,0.],  # idx 0
    [0.,0.,1.],  # idx 1
    [0.,1.,0.],  # idx 2
    [1.,0.,0.],  # idx 3
    [0.,1.,1.],  # idx 4
    [1.,0.,1.],  # idx 5
    [1.,1.,0.],  # idx 6
    [1.,1.,1.],  # idx 7
], dtype=tf.float32)


In [ ]:

# y = 1 if at least two inputs are 1, else 0
y_all = tf.cast(tf.reduce_sum(X_all, axis=1) >= 2.0, tf.float32)
y_all = tf.reshape(y_all, (-1, 1))


In [ ]:
print(X_all.shape)

In [ ]:
print(y_all.shape)

In [ ]:
print(y_all)

## 3) Train/Test split — hold out two unseen patterns

In [ ]:
train_idx = tf.constant([1,2,3,4,5,7])  # use 6 for training

In [ ]:
test_idx  = tf.constant([0,6])          # unseen: [0,0,0], [1,1,0]

In [ ]:
X_train = tf.gather(X_all, train_idx)

In [ ]:
y_train = tf.gather(y_all, train_idx)

In [ ]:
X_test  = tf.gather(X_all, test_idx)

In [ ]:
y_test  = tf.gather(y_all, test_idx)

In [ ]:
print(X_train.shape)

In [ ]:
print(X_test.shape)

## 4) Build a tiny model — 3 → hidden → 1 (sigmoid)

In [ ]:

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(units=3, activation='relu', input_shape=(3,)),
    Dense(units=1, activation='sigmoid'),
])


In [ ]:
model.summary()

## 5) Compile

In [ ]:

from tensorflow.keras.losses import BinaryCrossentropy
model.compile(optimizer='adam', loss=BinaryCrossentropy(), metrics=['accuracy'])


## 6) Train (few epochs; tiny data)

In [ ]:
history = model.fit(X_train, y_train, epochs=200, verbose=0)

In [ ]:
print(history.history['loss'][0])

In [ ]:
print(history.history['loss'][-1])

## 7) Evaluate on unseen data

In [ ]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)

In [ ]:
print(acc)

## 8) Predictions on the two unseen patterns

In [ ]:
pred_test = model.predict(X_test, verbose=0)

In [ ]:

import numpy as np
print(np.round(pred_test).astype(int).ravel().tolist())



### Reference truth table (for discussion)

| x₁ | x₂ | x₃ | y (majority ≥ 2) |
|---:|---:|---:|:----------------:|
| 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 0 |
| 0 | 1 | 0 | 0 |
| 1 | 0 | 0 | 0 |
| 0 | 1 | 1 | 1 |
| 1 | 0 | 1 | 1 |
| 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 1 |



## 9) Try it yourself (optional)

- Change the hidden layer units from 3 → 5 and re-run Section 6–8.
- Replace `relu` with `tanh` and observe any change in the final loss/accuracy.
- Train on a different 6-pattern split and keep 2 unseen; does it still generalize?
